In [ ]:
import numpy as np
import openai
import pandas as pd
import tiktoken
from dotenv import dotenv_values
from ratelimit import limits, sleep_and_retry
from docx import Document
from retry import retry

config = dotenv_values(".env")

openai.api_type = "azure"
openai.api_base = config["API_BASE"]
openai.api_version = config["API_VERSION"]
openai.api_key = config["API_KEY"]

completions_deployment_name = config["COMPLETIONS_DEPLOYMENT_NAME"]
embeddings_deployment_name = config["EMBEDDINGS_DEPLOYMENT_NAME"]

In [ ]:
encoding = tiktoken.get_encoding("gpt2") # encoding for text-davinci-003
contents = {
    'title': [],
    'section': [],
    'text': [],
    'tokens': []
}
section = ''
text = ''

doc = Document('./data/dtic_gameplan.docx')

for content in doc.paragraphs:

    if (content.style.name=='Heading 1' and \
         not "Customer & Engagement Overview" in content.text and \
         not "Proposed Architecture and Dev Plan" in content.text) or \
         content.style.name=='Heading 2':

        if content.text:
            if section:
                text = text.strip()
                tokens = len(encoding.encode(text))
                contents['title'].append('dtic_gameplan.docx')
                contents['section'].append(section)
                contents['text'].append(text)
                contents['tokens'].append(tokens)
            section = content.text
            text = ''

    else:
        text += f"{content.text} "

df = pd.DataFrame.from_dict(contents)
df = df.set_index(["title", "section"])

display(df)


In [ ]:
@retry(tries=3, delay=1)    # OpenAI seems to occasionally throw APIConnectionError for no good reason
@sleep_and_retry
@limits(calls=50, period=60)    # documented rate limit is 300/minute but... nope more like 50 <sigh>
def get_embedding(input: str, engine: str=embeddings_deployment_name) -> list[float]: # type: ignore
    result = openai.Embedding.create(
      engine=engine,
      input=input
    )
    return result["data"][0]["embedding"] # type: ignore

def compute_doc_embeddings(df: pd.DataFrame) -> dict[tuple[str, str], list[float]]:
    return {
        idx: get_embedding(r.text) for idx, r in df.iterrows()
    } # type: ignore


In [ ]:
document_embeddings = compute_doc_embeddings(df[(df['tokens'] > 0)])

In [ ]:
MAX_SECTION_LEN = 500
SEPARATOR = "\n* "
separator_len = len(encoding.encode(SEPARATOR))

def vector_similarity(x: list[float], y: list[float]) -> float:
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

def order_document_sections_by_query_similarity(query: str, contexts: dict[(str, str), np.array]) -> list[(float, (str, str))]: # type: ignore
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items() # type: ignore
    ], reverse=True)
    
    return document_similarities # type: ignore

def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame) -> str:
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections: # type: ignore
        # Add contexts until we run out of space.        
        document_section = df.loc[section_index]
        
        chosen_sections_len += document_section.tokens + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + document_section.text.replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    print(f"Selected {len(chosen_sections)} document sections:")
    print("\n".join(chosen_sections_indexes))
    
    header = """Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."\n\nContext:\n"""
    
    return header + "".join(chosen_sections) + "\n\n Q: " + question + "\n A:"



COMPLETIONS_API_PARAMS = {
    # We use temperature of 0.0 because it gives the most predictable, factual answer.
    "temperature": 0.0,
    "max_tokens": 500,
    "engine": completions_deployment_name
}

def answer_query_with_context(
    query: str,
    df: pd.DataFrame,
    document_embeddings: dict[(str, str), np.array], # type: ignore
    show_prompt: bool = False
) -> str:
    
    prompt = construct_prompt(
        query,
        document_embeddings,
        df
    )
    
    if show_prompt:
        print(prompt)

    response = openai.Completion.create(
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n") # type: ignore

In [ ]:
answer_query_with_context("What user personas exist in this project?", df, document_embeddings)

In [ ]:
answer_query_with_context("What Azure resources will use private endpoints?", df, document_embeddings)